<a href="https://colab.research.google.com/github/fadelnast/Testing/blob/master/Copy_of_UjianDTSProjectRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#0. Import Library

In [0]:
# tenserflow sebagai deep learning proses
import tensorflow as tf 
from tensorflow.contrib import rnn
import numpy as np # untuk mendefinisikan array
import pandas as pd # untuk mendefinisikan dataframe agar lebih complex
import time # waktu
# metplotlib sebagai ploting data
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
%matplotlib inline
mpl.style.use( 'ggplot' ) # style Plot dari matplotlib tsb

ModuleNotFoundError: ignored

#1. Mengenal Data

In [0]:
# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
# mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)
mnist = input_data.read_data_sets('data/fashion', one_hot=True) # membaca dataset dari input_data yang berada apada tensorflow.examples.tutorial.mnist


ModuleNotFoundError: ignored

In [0]:
!mkdir data/fashion # membuat sebuah directory dalam input_data yang bernama "data/fashion"
!wget -O data/fashion/train-images-idx3-ubyte.gz http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz
!wget -O data/fashion/train-labels-idx1-ubyte.gz http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-labels-idx1-ubyte.gz
!wget -O data/fashion/t10k-images-idx3-ubyte.gz http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-images-idx3-ubyte.gz
!wget -O data/fashion/t10k-labels-idx1-ubyte.gz http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-labels-idx1-ubyte.gz

In [0]:
# Import Fashion MNIST
mnist = input_data.read_data_sets('data/fashion', one_hot=True) # memanggil sebuah data pada dataset untuk mengextrak data


> Membaca dataset fashion mnist





In [0]:
print(f'image training: { mnist.train.images.shape}')
print(f'label training: { mnist.train.labels.shape}')
print(f'image testing: { mnist.test.images.shape}')
print(f'label testing: { mnist.test.labels.shape}')


In [0]:
print(mnist.train.images[40]) #membaca data images ke 40

In [0]:
sample_1 = mnist.train.images[40].reshape(28,28) # untuk memanggil image ke 40 dengan ukuran pixel 28x28
plt.imshow(sample_1, cmap='Greys') #menampilkan inputan dari variable sample_i dengan memberikan warna abu-abu

In [0]:
datatrainimages=pd.DataFrame(mnist.train.images) # untuk meengubah data dari numpy ke data frame pandas
datatestimages=pd.DataFrame(mnist.test.images)
combine = [datatrainimages, datatestimages] 
ffmnist = pd.concat(combine)
print(ffmnist)
print(ffmnist.isnull().any().sum()) # untuk menghitung jumlah data yang kosong 

In [0]:
fig, axes = plt.subplots(2, 5, figsize=(7,4))
for img, label, ax in zip(mnist.train.images[0:10], np.argmax(mnist.train.labels[0:10], 1), axes.flat):
  ax.set_title(label)
  ax.imshow(img.reshape(28,28))
  ax.axis('off')
plt.show()

#2. Membangun RNN/LSTM

##2.1. Hyperparameter

In [0]:
# 55000
#hyperparamters
learning_rate = 0.0001
# jumlah perpndaha
training_iterations = 100000
# banyaknya pengulangan
batch_size = 128
# banyaknya jumlah gambar yang diolah dalam sekali pengolahan
display_step = 10
# 
epochs = 1000
# 

n_inputs = 28
# jumlah neuran masukan
n_time_steps = 28
# 
n_hidden_units = 128
# 
n_classes = 10
# 

##2.2. Placeholders

In [0]:
X = tf.placeholder(tf.float32, [None, n_time_steps, n_inputs])
# X = tf.placeholder(tf.float32, [batch_size, n_time_steps, n_inputs])
Y = tf.placeholder(tf.float32, [None, n_classes])
X

##2.3. Variable

In [0]:
output_w = tf.Variable(tf.random_normal([n_hidden_units, n_classes]))
output_b = tf.Variable(tf.random_normal([n_classes]))
output_w

##2.4. Menyiapkan dan Membangun RNN/LSTM

In [0]:
lstm_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden_units, forget_bias=1)
outputs, final_state = tf.nn.dynamic_rnn(lstm_cell, X, dtype="float32")
outputs

In [0]:
outputs[:,-1,:]

In [0]:
# a = np.array([[[1,2,3], [4,5,6], [7,8,9]], [[10,11,12], [13,14,15], [16,17,18]], [[19,20,21], [22,23,24], [25,26,27]]]) # menyimpan hasil pixel image 28 x 28
# a

In [0]:
# a[:,-1,:]

##2.5. Memprediksi

In [0]:
logits = tf.matmul(outputs[:,-1,:], output_w) + output_b #(outputs[:,-1,:]*output_w) + output_b 
logits

##2.6. Loss dan Optimasi

In [0]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits )) #mencari jumlah rata rata dari cross and tropy dari label = y dan logits

optimize = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss) # pengoptimasikan leartning_rate dengan jumlah lose menggunakan AdamOptimizer
# optimize = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss) 
# opsi lainnya

##2.7. Metric untuk evaluasi

In [0]:
# pembuatan konfusion matrik yang digunakan untuk mengoptimasikan
confusion_op = tf.math.confusion_matrix(tf.argmax(Y, 1), tf.argmax(logits, 1)) 
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1)) 
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

#3. Training

In [0]:
timestart=time.time()
# insialisasi variable
init = tf.global_variables_initializer()
graph=[]

# inisialisasi seasino(untuk menjalankan graph)
session = tf.Session()
session.run(init)

for epoch in range(epochs):
  # mengambil gambar gambar dalam batch
  batch_x, batch_y = mnist.train.next_batch(batch_size = batch_size)
  
  # reshape, karena gambarnya masih dalam bentuk vektor
  batch_x2 = batch_x.reshape((batch_size, n_time_steps, n_inputs))
  
  session.run(optimize, feed_dict={X: batch_x2, Y: batch_y})
  
  if not epoch % 10: # hanya menampilkan stap ke 10
    
    acc = session.run(accuracy, feed_dict={X: batch_x2, Y: batch_y})
    los = session.run(loss, feed_dict={X: batch_x2, Y: batch_y})
    graph.append([epoch,acc,los]) # graph menyimpan sebuah data dari epoch,acc dan loss untuk digunakan pada ploting
    print(f'epoch: {epoch}, loss: {los:.4f}, accuracy: {acc:.4f}')
    # print(graph)
    
  if epoch==(epochs-1):
    
    output_val = np.array(session.run(outputs, feed_dict={X: batch_x2}))
    finalstate_val = np.array(session.run(final_state, feed_dict={X: batch_x2}))
timestop=time.time()

In [0]:
print(timestop-timestart) # menampilkan waktu proses training

#4. Evaluasi

In [0]:
confusion = session.run(confusion_op, feed_dict={X: mnist.test.images.reshape(10000, n_time_steps, n_inputs), Y: mnist.test.labels})
print(confusion)

In [0]:
from sklearn.metrics import classification_report

yhat = session.run(logits, feed_dict={X: mnist.test.images.reshape(10000, n_time_steps, n_inputs)})
yhat = np.argmax(yhat, axis=1)

y = session.run(Y, feed_dict={Y: mnist.test.labels})
y = np.argmax(y, axis=1)

print(classification_report(y, yhat))

In [0]:
# untuk menampilkan data presentasi dari setiap data test
i=0 
presentasi_data = session.run(logits, feed_dict={X: mnist.test.images.reshape(10000, n_time_steps, n_inputs)})
while i < len(y):
  print ("Data test ke",i)
  print (y[i],yhat[i])
  print (presentasi_data[i])
  i=i+1

In [0]:
target_dict = {
 0: 'T-shirt/top',
 1: 'Trouser',
 2: 'Pullover',
 3: 'Dress',
 4: 'Coat',
 5: 'Sandal',
 6: 'Shirt',
 7: 'Sneaker',
 8: 'Bag',
 9: 'Ankle boot',
}

In [0]:
fig, axes = plt.subplots(10, 10, figsize=(12,14))
for img, label, ax in zip(mnist.test.images[0:100], yhat, axes.flat):
  ax.set_title(target_dict[label])
  ax.imshow(img.reshape(28,28))
  ax.axis('off')
plt.show()

In [0]:
# plot antara epoch dan akurasi
sumbux=[item[0] for item in graph]
sumbuy=[item[1] for item in graph]
# print(sumbuy)
plt.plot(sumbux, sumbuy)
plt.yscale('linear')
plt.title('akurasi')
plt.xlabel('epoch')
plt.ylabel('akurasi')
plt.axis('auto')
plt.grid(True)

In [0]:
# plot antara epoch dan loss
sumbux=[item[0] for item in graph]
sumbuy=[item[2] for item in graph]
plt.plot(sumbux, sumbuy)
# print(sumbuy)
plt.yscale('linear')
plt.title('loss')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.axis('auto')
plt.grid(True)